In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Creating Spark Session
sp = SparkSession.builder.appName("Customer").getOrCreate()

In [ ]:
df_cc = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

df_cust = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CUSTOMER") \
    .option("user", "root") \
    .option("password", "password") \
    .load()
df_br = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_BRANCH") \
    .option("user", "root") \
    .option("password", "password") \
    .load()
    

In [ ]:
df_cc.show()

In [ ]:
df_cust.show(5)

In [ ]:

df_join = df_cc.join(df_cust, df_cc.CUST_SSN == df_cust.SSN, 'inner').select(col('SSN'), col('CUST_ZIP'), col('TIMEID'), \
      col('TRANSACTION_ID'), col('TRANSACTION_VALUE'))




In [ ]:
df_join.show()

In [ ]:
df_join.createOrReplaceTempView('CC_CUST')

In [ ]:
data = sp.sql('select * from CC_CUST where CUST_ZIP == "53066" and substr(TIMEID,1,4) == "2018" and substr(TIMEID,5,2)=="06"')
data.show(100)

Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.

In [ ]:
def trans_value(zip,yr,mm):
    print("inside the function", zip, yr, mm)
    data = sp.sql('select distinct * from CC_CUST where CUST_ZIP == "{}" and substr(TIMEID,1,4) == "{}" \
        and substr(TIMEID,5,2) == "{}" ORDER BY substr(TIMEID,7,2) DESC'.format(zip,yr,mm))
    data.show()


zip = str(input("enter zipcode: "))
yr = str(input("enter year: "))
mm = str(input("enter month: "))
trans_value(zip,yr,mm)

2.. Used to display the number and total values of transactions for a given type.

In [ ]:
#using df
df_cc.select('transaction_type','transaction_value').groupby('transaction_type').agg(count('transaction_type'), \
    sum('transaction_value')).show()


In [ ]:

#using view
df_cc.createOrReplaceTempView('cdw_cc')

In [ ]:
data1 = sp.sql('SELECT transaction_type, COUNT(*) as trans_count, \
                round(SUM(transaction_value),2) as trans_val \
                FROM cdw_cc \
                GROUP BY transaction_type')


# 2.3 Used to display the number and total values of transactions for branches in a given state.

In [46]:
data_br = df_cc.join(df_br, df_br.BRANCH_CODE == df_cc.BRANCH_CODE, 'inner').select(df_br['branch_code'].alias('BC1'), \
            col('branch_name'), col('branch_state'), col('transaction_value'))
data_br.show()

+---+------------+------------+-----------------+
|BC1| branch_name|branch_state|transaction_value|
+---+------------+------------+-----------------+
| 26|Example Bank|          TX|            56.12|
| 29|Example Bank|          CA|            17.03|
| 26|Example Bank|          TX|            25.01|
| 29|Example Bank|          CA|            86.18|
| 29|Example Bank|          CA|            66.37|
| 26|Example Bank|          TX|            20.14|
| 26|Example Bank|          TX|            86.87|
| 26|Example Bank|          TX|            20.04|
| 29|Example Bank|          CA|            34.93|
| 26|Example Bank|          TX|            20.87|
| 26|Example Bank|          TX|             80.3|
| 29|Example Bank|          CA|           100.81|
| 29|Example Bank|          CA|            17.76|
| 26|Example Bank|          TX|            58.56|
| 26|Example Bank|          TX|            70.19|
| 29|Example Bank|          CA|            72.88|
| 26|Example Bank|          TX|            12.78|


In [47]:
data_br.createOrReplaceTempView('data_br_view')


In [48]:
def trans_by_branch(state):
    data1 = sp.sql("SELECT bc1, COUNT(bc1), \
                round(SUM(transaction_value),2) \
                FROM data_br_view \
                WHERE BRANCH_STATE == '{}' \
                GROUP BY bc1 \
                ORDER BY bc1".format(state))
    data1.show()

state = str(input("Enter State : "))
trans_by_branch(state)

+---+----------+--------------------------------+
|bc1|count(bc1)|round(sum(transaction_value), 2)|
+---+----------+--------------------------------+
|  1|       374|                        18157.57|
| 20|       381|                        19350.83|
|124|       372|                        18644.89|
+---+----------+--------------------------------+

